In [3]:
import spotipy,os
import spotipy.util as util
import json

client_id = 'a5f4717d6cbe43fea2cc354da04490b6'
redirect_uri = 'http://localhost:8888/callback/'
client_secret = '4eaba7b3f548455dae6a2ac83d8e39e1'
scope = ['user-library-read','user-read-private','user-read-recently-played','user-read-currently-playing']


Oauth = spotipy.oauth2.SpotifyOAuth(client_id=client_id,
                            client_secret=client_secret,
                            redirect_uri=redirect_uri,
                            scope=scope)
token = Oauth.get_access_token()
print(token)
headers = {"Authorization": "Bearer {}".format(token['access_token'])}

sp = spotipy.client.Spotify(headers)
data = sp.current_user_recently_played(50)

items =  data['items']
print(items)

data = find_songs_name(items)



C:\Users\kevin\AppData\Local\Temp\ipykernel_8128\39353376.py:15: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = Oauth.get_access_token()


{'access_token': 'BQBvTs4jWvfslf-mTgkQyvEpOiReueJNNrZHSDfxr4qnXMf7axUFl6avhfZO5K-R6QYc0TbzvY9uEuE4XIDd9DAEavVgffUv_disJWM3bJYiUY0AYW7la1QS4M1OnFadqG3w1Gee75db1XsflWl0DVquE0S9D0w0LlErpMdZ2AXi4x8LxndNm7eenRJjUwm3m6zn0rjWalrlhJba', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQDROefR5cOiwNcyosQ5lSrWWm1mSOTpAMYEueWCc4Sm4vwphUW0P6oNQQlNSf36jp5NytzbAfN_tjRqji3Ik3FB493fTqXUaoar1pUmqEpOiwupj7aS_PUmC-KFu2oIQSM', 'scope': 'user-library-read user-read-currently-playing user-read-private user-read-recently-played', 'expires_at': 1705404240}
[{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4L0gz16xuM1as1OgYzh1SR'}, 'href': 'https://api.spotify.com/v1/artists/4L0gz16xuM1as1OgYzh1SR', 'id': '4L0gz16xuM1as1OgYzh1SR', 'name': 'Kensuke Ushio', 'type': 'artist', 'uri': 'spotify:artist:4L0gz16xuM1as1OgYzh1SR'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', '

In [24]:
count_song(data)

[['One Summer Day',
  'Joe Hisaishi',
  'https://i.scdn.co/image/ab67616d00001e02fd220b07117400ccbc85ed96',
  1],
 ['Wyoming',
  'Elijah Fox',
  'https://i.scdn.co/image/ab67616d00001e02f4923095b4e0b0dcc5dc9bc6',
  1],
 ['(Only) About Love - Demo',
  'grentperez',
  'https://i.scdn.co/image/ab67616d00001e02deb94b2957cbe7429a23f85c',
  1],
 ['Dreaming With You',
  'The Chairs',
  'https://i.scdn.co/image/ab67616d00001e02d601c5d833eb8f262d324b91',
  1],
 ['omoide cafe',
  'Ichiko Aoba',
  'https://i.scdn.co/image/ab67616d00001e02cec430c23bc8f46203eb93db',
  1],
 ['pochette no outa',
  'Ichiko Aoba',
  'https://i.scdn.co/image/ab67616d00001e02c273a69726da08e6b6f30927',
  1],
 ['Like the Movies',
  'Laufey',
  'https://i.scdn.co/image/ab67616d00001e02afe473a4a47a4e69ab174069',
  1],
 ['I Wish You Love',
  'Laufey',
  'https://i.scdn.co/image/ab67616d00001e02afe473a4a47a4e69ab174069',
  1],
 ['gentle ending',
  'Kensuke Ushio',
  'https://i.scdn.co/image/ab67616d00001e02ad94c34cdc0e6a2a3314

In [23]:
def find_songs_name(items):
    last50songs = []
    for i in range(len(items)):
        data = []
        current_track = items[i]['track']
        time = items[i]['played_at']
        album = current_track['album']
        album_name = album['name']
        album_image_inf = album['images'][1]
        album_image_url = album_image_inf['url']
        artist_inf = album['artists']
        artist_name = artist_inf[0]['name']

        t = trans_time(time)
        # print(str(t))

        data.append(current_track['name'])
        data.append(artist_name)
        data.append(album_image_url)
        data.append(t)
        last50songs.append(data)

    return last50songs
    
def take_second(list):
    return list[2]

def trans_time(t):
    t = t.split('T')
    date = t[0]
    time = t[1].split('.')[0]
    return date+" "+time

def get_data():

    token = get_token()
    headers = {"Authorization": "Bearer {}".format(token['access_token'])}

    sp = spotipy.client.Spotify(headers)
    data = sp.current_user_recently_played(50)

    items =  data['items']

    last50songs = find_songs_name(items)

    return last50songs

def count_song(data):
    song_count = {}
    song_list = []
    for song in data:
        # print(song)
        song_name = song[0]
        artist_name =  song[1]
        song_image_url = song[2]
        if song_name in song_count:
            continue
        else:
            song_count.setdefault(song_name,[])
            song_count[song_name].append(0)
            song_count[song_name].append(artist_name)
            song_count[song_name].append(song_image_url)

    for song in data:
        song_name = song[0]
        song_count[song_name][0] +=1

    for item in song_count:
        dic = {}
        # print(item,song_count[item][0],song_count[item][1],song_count[item][2])
        dic['SongName'] = item
        dic['Artist'] = song_count[item][1]
        dic['Cover'] = song_count[item][2]
        dic['Count'] = song_count[item][0]
        data = [item, song_count[item][1] ,song_count[item][2],song_count[item][0]]
        song_list.append(data)


    song_list.sort(key=take_second,reverse=True)
    
    return song_list


def get_user_info():
    token = get_token()
    headers = {"Authorization": "Bearer {}".format(token['access_token'])}
    sp = spotipy.client.Spotify(headers)
    user = sp.me()
    l = []
    l.append(user['display_name'])
    l.append(user['images'][0]['url'] )

    return l
